In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import json
from langfuse import Langfuse
from langfuse.decorators import observe, langfuse_context
from litellm import completion
import litellm
import os
from nemoguardrails import LLMRails, RailsConfig
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
# from models.ChatOpenRouter import ChatOpenRouter
from models.ChatOpenRouter import ChatOpenRouter

from nemoguardrails.llm.providers import register_llm_provider
import nest_asyncio

load_dotenv()

# Initialize Langfuse client
langfuse = Langfuse()
litellm.success_callback = ["langfuse"] # log input/output to lunary, mlflow, langfuse, helicone
# config_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), "config", "config.yml")
config_path = "/Users/piyushkumarjain/Projects/github/genai/rag-guardrails-app/app/config/config.yml"
print(f"Loading config from: {config_path}")
if not os.path.exists(config_path):
    raise FileNotFoundError(f"Config file not found at {config_path}")

config = RailsConfig.from_path(config_path)# Configure LiteLLM

# Load FAISS index and documents
model = SentenceTransformer('all-MiniLM-L6-v2')


# openrouter_model = ChatOpenRouter(
#     model_name="openrouter/qwen/qwen3-30b-a3b:free",
# )

openrouter_provider = ChatOpenRouter(
    model=os.environ.get("OPENROUTER_MODEL"),
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)

register_llm_provider("custom_llm", ChatOpenRouter)


try:
    
    # Load FAISS index
    index = faiss.read_index("../data/vector_index.faiss")
    
    # Load documents from JSON
    with open("../data/documents.json", "r", encoding="utf-8") as f:
        documents = json.load(f)
    
    if not isinstance(documents, list):
        raise ValueError("Documents JSON must be a list")
        
except FileNotFoundError as e:
    raise RuntimeError("Required files not found. Please ensure:")
    raise RuntimeError("1. data/vector_index.faiss exists")
    raise RuntimeError("2. data/documents.json exists and is valid JSON")

@observe(as_type="generation")
async def rag_pipeline(query):
    query_vector = model.encode([query])
    _, indices = index.search(query_vector, 2)
    retrieved_docs = [documents[i] for i in indices[0]]
    messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Answer the question based on the provided context."
    },
    {
        "role": "user",
        "content": f"Context: {retrieved_docs}\n\nQuestion: {query}"
    }
]
    langfuse_context.update_current_observation(
        input=messages,
        model = os.environ.get("OPENROUTER_MODEL")
)

    try:
        nest_asyncio.apply()
        app = LLMRails(config,llm=openrouter_model, verbose=False)

        response = app.generate(
                                 messages=[{"role": "user", "content": "What is the capital of France?"}])
        # response = completion(
        #             model=os.environ.get("OPENROUTER_MODEL"),
        #             messages=messages,
        #             api_key=os.environ.get("OPENROUTER_API_KEY")
        #         )
        print(response)
        answer = response.choices[0].message.content.strip()

        langfuse_context.update_current_observation(
                usage_details={
                    "input": response.usage.prompt_tokens,
                    "output": response.usage.completion_tokens
                })
        return answer
    except Exception as e:
        raise e



/Users/piyushkumarjain/miniconda3/envs/rag-demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Loading config from: /Users/piyushkumarjain/Projects/github/genai/rag-guardrails-app/app/config/config.yml


In [ ]:
llm = ChatOpenAI(
                        model="sao10k/l3-lunaris-8b",
                        api_key="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",
                        base_url="https://openrouter.ai/api/v1"
                    )

In [3]:

llms = {
    "main": ChatOpenRouter(
        api_key="your_api_key",
        model="openrouter/qwen/qwen3-30b-a3b:free"
    )
}

In [4]:
# register_llm_provider("custom_llm", ChatOpenRouter)

app = LLMRails(
    config=config,
    llm=openrouter_provider,
    verbose=True,
)

Entered verbose mode.

23:43:01.048 | Registered Actions ['GetAttentionPercentageAction', 'GetCurrentDateTimeAction', 
'UpdateAttentionMaterializedViewAction', 'alignscore request', 'alignscore_check_facts', 
'autoalign_factcheck_output_api', 'autoalign_groundedness_output_api', 'autoalign_input_api', 
'autoalign_output_api', 'call cleanlab api', 'call fiddler faithfulness', 'call fiddler safety on bot message', 
'call fiddler safety on user message', 'call gcpnlp api', 'call_activefence_api', 'content_safety_check_input', 
'content_safety_check_output', 'create_event', 'detect_pii', 'detect_sensitive_data', 
'jailbreak_detection_heuristics', 'jailbreak_detection_model', 'llama_guard_check_input', 
'llama_guard_check_output', 'mask_pii', 'mask_sensitive_data', 'patronus_api_check_output', 
'patronus_lynx_check_output_hallucination', 'protect_text', 'retrieve_relevant_chunks', 'self_check_facts', 
'self_check_hallucination', 'self_check_input', 'self_check_output', 'summarize_document', 
'topic_safety_check_input', 'wolfram alpha request']

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 113359.57it/s]


In [5]:
import nest_asyncio


In [7]:
nest_asyncio.apply()
app.generate(
    messages=[{"role": "user", "content": "What is the capital of France?"}]
)

23:43:09.672 | Event UtteranceUserActionFinished | {'final_transcript': 'What is the capital of France?'}

23:43:09.673 | Event StartInternalSystemAction | {'uid': '8c72...', 'action_name': 'create_event', 'action_params':
{'event': {'_type': 'StartInputRails'}}, 'action_result_key': None, 'action_uid': 'c977...', 'is_system_action': 
True}

23:43:09.674 | Executing action create_event

23:43:09.675 | Event StartInputRails | {'uid': 'cba6...'}

23:43:09.677 | Event StartInternalSystemAction | {'uid': '7db4...', 'action_name': 'create_event', 'action_params':
{'event': {'_type': 'StartInputRail', 'flow_id': '$triggered_input_rail'}}, 'action_result_key': None, 
'action_uid': '8669...', 'is_system_action': True}

...

23:43:11.265 | Event UserMessage | {'uid': '436a...', 'text': 'What is the capital of France?'}

23:43:11.268 | Event StartInternalSystemAction | {'uid': 'ee88...', 'action_name': 'generate_user_intent', 
'action_params': {}, 'action_result_key': None, 'action_uid': '4f24...', 'is_system_action': True}

23:43:11.269 | Executing action generate_user_intent

Payload: {'model': 'google/gemini-2.0-flash-exp:free', 'messages': [{'role': 'user', 'content': 'Below is a conversation between a helpful AI assistant and a user. The bot is designed to generate human-like text based on the input that it receives. The bot is talkative and provides lots of specific details. If the bot does not know the answer to a question, it truthfully says it does not know.\n\nUser: What is the capital of France?\nAssistant:'}], 'temperature': 0.7, 'max_tokens': 256}
Response: <ClientResponse(https://openrouter.ai/api/v1/chat/completions) [200 OK]>
<CIMultiDictProxy('Date': 'Wed, 30 Apr 2025 15:43:11 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'x-clerk-auth-message': 'Invalid JWT form. A JWT consists of three parts separated by dots. (reason=token-invalid, token-carrier=header)', 'x-clerk-auth-reason': 'token-invalid', 'x-clerk-auth-status': 'signed-out', 'Vary': 'Accept-En

LLMCallException: LLM Call Exception: 'choices'